In [15]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import time
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import cornac
import surprise

| Method                         | RMSE | MAP@K | NDCG@K | Novelty | Diversity |
|--------------------------------|------|-------|--------|---------|-----------|
| User-based                     | 0.98 | NaN   | 0.89   | -5.41   | 0.78      |
| Item-based                     | 0.00 | NaN   | 0.00   | 0.00    | 0.00      |
| Clustering                     | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| Clustering + Z-score           | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| KNNregression (classification) | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| MF                             | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| Surprise_MF                    | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| Cornac_NN                      | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |

User- Item based решения не имеют (имеют 1.00) MAP@K, так как предсказание идет только для конкретных фильмов из test. Иначе предсказание всей матрицы занимает несколько часов из-за поэлементного вычисления

In [34]:
# Функции для метрик
def calculate_rmse(df):
    return np.sqrt(mean_squared_error(df['rating'], df['predicted_rating']))

def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual: #and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k)

def calculate_mapk(df, k=10):
    user_group = df.groupby('userId')
    apk_scores = []
    for user, group in user_group:
        actual = group['movieId'].values
        predicted = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values
        apk_scores.append(apk(actual, predicted, k))
    return np.mean(apk_scores)

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0.0

def ndcg_at_k(actual, predicted, k=10):
    predicted = predicted[:k]
    ideal = sorted(actual, reverse=True)[:k]
    return dcg_at_k(predicted, k) / dcg_at_k(ideal, k)

def calculate_ndcgk(df, k=10):
    user_group = df.groupby('userId')
    ndcg_scores = []
    for user, group in user_group:
        actual = group['rating'].values
        predicted = group.sort_values(by='predicted_rating', ascending=False)['rating'].values
        ndcg_scores.append(ndcg_at_k(actual, predicted, k))
    return np.mean(ndcg_scores)

def calculate_novelty(df, item_popularity, k=10):
    count_users = len((df['userId'].unique()))
    user_group = df.groupby('userId')
    novelty_scores = []
    for user, group in user_group:
        recommended_items = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values[:k]
        novelty = np.mean([np.log2(1 / item_popularity[item]) for item in recommended_items if item in item_popularity])
        novelty_scores.append(novelty)
    return np.mean(novelty_scores)

def calculate_diversity(df, item_similarity_matrix, k=10, matrix_columns=None):
    user_group = df.groupby('userId')
    diversity_scores = []
    
    for user, group in user_group:
        recommended_items = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values[:k]
        total_pairs = 0
        diversity = 0
        for i in range(len(recommended_items)):
            for j in range(i + 1, len(recommended_items)):
                # Получаем индексы фильмов в матрице
                if recommended_items[i] in matrix_columns and recommended_items[j] in matrix_columns:
                    idx_i = matrix_columns.get_loc(recommended_items[i])
                    idx_j = matrix_columns.get_loc(recommended_items[j])
                    total_pairs += 1
                    diversity += (1 - item_similarity_matrix[idx_i, idx_j])
        
        if total_pairs > 0:
            diversity_scores.append(diversity / total_pairs)
    
    return np.mean(diversity_scores)

610

In [17]:
def create_cf_matrix(train, cf_type='user'):

    """
    Создает матрицу для коллаборативной фильтрации.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - Матрица для коллаборативной фильтрации.
    """
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating')
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating')
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")

In [29]:
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test, df

In [33]:
# 2-3. User-based and Item-based коллаборативная фильтрация с/без z-score
def collaborative_filtering(train, test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=None):
    # Используем отдельную функцию для создания матрицы
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Вычисляем косинусное сходство
    similarity = cosine_similarity(matrix.fillna(0))
    
    # Вычисляем средние и стандартные отклонения для Z-score (если нужно)
    means = matrix.mean(axis=1)
    stds = matrix.fillna(0).std(axis=1) if use_z_score else None

    # Если включен Z-score, нормализуем рейтинги
    if use_z_score:
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0).fillna(0)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            # Если тип фильтрации пользователь-фильм
            user_index, item_index = user_id, item_id
        else:
            # Если тип фильтрации фильм-пользователь
            user_index, item_index = item_id, user_id
        
        # Если пользователь или элемент отсутствует в базе
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Рейтинги текущего пользователя или фильма
        ratings = matrix.loc[user_index]
        # Сходства с другими пользователями или фильмами
        similarities = similarity[matrix.index.get_loc(user_index)]
        
        # Сортируем по схожести и выбираем топ-N (если задано)
        sorted_similarities = np.argsort(similarities)[::-1]
        top_similar = sorted_similarities[:N] if N else sorted_similarities
        
        numerator = 0
        denominator = 0
        for similar in top_similar:
            # Проверяем, есть ли рейтинг для указанного элемента (пользователя или фильма)
            if matrix.iloc[similar][item_index] > 0:
                if use_z_score:
                    # Коррекция на Z-score
                    adjusted_rating = matrix_normalized.iloc[similar][item_index]
                    numerator += similarities[similar] * adjusted_rating
                elif use_mean_adjustment:
                    # Коррекция на среднюю оценку
                    adjusted_rating = matrix.iloc[similar][item_index] - means.iloc[similar]
                    numerator += similarities[similar] * adjusted_rating
                else:
                    numerator += similarities[similar] * matrix.iloc[similar][item_index]
                
                # Знаменатель: \text{denominator} += \text{sim}(u, v)
                denominator += abs(similarities[similar])
        
        # Если нет похожих пользователей/фильмов, предсказываем средний рейтинг пользователя
        if denominator == 0:
            return ratings.mean()
        
        if use_z_score:
            # Предсказание через Z-score
            predicted_z_score = numerator / denominator
            return means.loc[user_index] + predicted_z_score * stds.loc[user_index]
        elif use_mean_adjustment:
            # Предсказание с коррекцией на среднее значение
            return means.loc[user_index] + numerator / denominator
        else:
            # Предсказание без коррекции на среднее значение
            return numerator / denominator

    # Предсказываем рейтинги только для тех, которых не было в train
    test['predicted_rating'] = test.apply(
        lambda row: predict_rating(row['userId'], row['movieId']), 
        axis=1
    )

    # Вычисляем метрики на отфильтрованных данных
    rmse = calculate_rmse(test)
    mapk = calculate_mapk(test, k=10)
    ndcgk = calculate_ndcgk(test, k=10)
    
    item_popularity = train['movieId'].value_counts().to_dict()
    item_similarity_matrix = 1 - pairwise_distances(matrix.T.fillna(0), metric='cosine')
    novelty = calculate_novelty(test, item_popularity, k=10)
    diversity = calculate_diversity(test, item_similarity_matrix, k=10, matrix_columns=matrix.columns)
    
    return {
        'RMSE': rmse,
        'MAP@K': mapk,
        'NDCG@K': ndcgk,
        'Novelty': novelty,
        'Diversity': diversity
    }

In [40]:
train, test, df = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.3)

user = collaborative_filtering(train, test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=None)
print(user)

{'RMSE': 0.9765590439849144, 'MAP@K': 1.0, 'NDCG@K': 0.8862164437146262, 'Novelty': -5.407149403543376, 'Diversity': 0.7842086565702061}


In [ ]:
item = collaborative_filtering(train, test, cf_type='item', use_mean_adjustment=False, use_z_score=False, N=None)
print(item)

In [ ]:
# 4-5. Рекомендательная система на основе кластеризации и расчета средней оценки кластера
def cluster_based_cf(train, test, n_clusters=40, use_z_score=False, cf_type='user'):
    """
    Кластерная коллаборативная фильтрация.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - n_clusters: количество кластеров для K-means.
    - use_z_score: флаг для использования Z-score нормализации.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу с помощью create_cf_matrix
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    if use_z_score:
        # Нормализуем оценки с помощью z-score
        means = matrix.mean(axis=1)
        stds = matrix.fillna(0).std(axis=1)
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0)
        matrix_for_clustering = matrix_normalized.fillna(0)
    else:
        # Используем обычную матрицу для кластеризации
        matrix_for_clustering = matrix.fillna(0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(matrix_for_clustering)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id
        
        # Проверка, если пользователя или элемента нет в матрице
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Определяем кластер пользователя или фильма
        cluster = clusters[matrix.index.get_loc(user_index)]
        cluster_items = matrix.index[clusters == cluster]
        
        if use_z_score:
            # Получаем нормализованные рейтинги для кластера
            cluster_ratings_normalized = matrix_normalized.loc[cluster_items, item_index]
            
            # Проверяем, пусты ли нормализованные рейтинги
            if cluster_ratings_normalized.empty or cluster_ratings_normalized.isna().all():
                return matrix.mean().mean()  # Если нет рейтингов, вернуть среднее по всей матрице
            
            predicted_z_score = cluster_ratings_normalized.mean()
            # Преобразуем предсказание обратно в исходные единицы
            return means[user_index] + (predicted_z_score * stds[user_index])
        else:
            # Получаем обычные рейтинги для кластера
            cluster_ratings = matrix.loc[cluster_items, item_index]
            
            # Проверка на наличие не NaN значений
            if cluster_ratings.empty or cluster_ratings.isna().all():
                return matrix.mean().mean()  # Если нет доступных значений, вернуть среднее по всей матрице
            
            # Возвращаем среднее значение по кластеру
            return cluster_ratings.mean()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse